In [1]:
import pandas as pd
import os
import json
import tiktoken
import openai
import numpy as np
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage

In [3]:
BASE_URL = "https://danijel-test02.openai.azure.com/"
API_KEY = "05fe89f89e984fd188cb6143d6f5699a"
DEPLOYMENT_NAME = "gpt4"
model = AzureChatOpenAI(
    openai_api_base=BASE_URL,
    openai_api_version="2023-05-15",
    deployment_name=DEPLOYMENT_NAME,
    openai_api_key=API_KEY,
    openai_api_type="azure",
)

In [4]:
data = pd.read_excel('https://domenstorageacc.blob.core.windows.net/randomstuff/Kategorije_delovna%20verzija_BS_20092021.xlsx', skiprows=2)
data.head()

,ASISTENCA/INFO,VRSTA STORITVE - Področje asistence oz. informacij,VSEBINA VPRAŠANJA,Unnamed: 3,STRANKINA VPRAŠANJA Kaj stranke vprašajo pri določeni temi; Kaj jih zanima?,EXTERNAL ODGOVOR (STRANKI),HTML EXTERNAL,INTERNAL NAVODILA ZAPOSLENI,HTML INTERNAL,KLJUČNE BESEDE,DRUGI NIVO (DA/NE),KDO,NASLOVI
0,Informacije,Sprememba kontaktnih podatkov,Sprememba kontaktnih podatkov,NaN,Kako uredim spremembo imena/priimka?,Za spremembo osebnih podatkov je potrebno odda...,NaN,"Stranki pojasnimo, da lahko vlogo za spremembo...",NaN,"ime, priimek, osebni podatek, kontakt, zahteve...",Da,"Zaledna podpora, Podpora ŽZ, TZZ, Triglav Poko...",NaN
1,Informacije,Sprememba kontaktnih podatkov,Sprememba kontaktnih podatkov,NaN,Kako uredim spremembo kontaktnega podatka?,Za spremembo kontaktnih podatkov je potrebno o...,NaN,"Stranki pojasnimo, da lahko vlogo za spremembo...",NaN,"osebni podatek, kontakt, zahtevek, sprememba",Da,"Zaledna podpora, Podpora ŽZ, TZZ, Triglav Poko...",NaN
2,Informacije,Sprememba kontaktnih podatkov,Sprememba kontaktnih podatkov,NaN,Kako uredim spremembo telefonske številke?,Za spremembo osebnih podatkov je potrebno odda...,NaN,"Stranki pojasnimo, da lahko vlogo za spremembo...",NaN,"telefonska številka, telefon, osebni podatek, ...",Da,"Zaledna podpora, Podpora ŽZ, TZZ, Triglav Poko...",NaN
3,Informacije,Sprememba kontaktnih podatkov,Sprememba kontaktnih podatkov,NaN,Kako uredim spremembo e-maila/e-poštnega naslova?,Za spremembo osebnih podatkov je potrebno odda...,NaN,"Stranki pojasnimo, da lahko vlogo za spremembo...",NaN,"e-mail, e-pošta, e-naslov, kontakt, zahtevek, ...",Da,"Zaledna podpora, Podpora ŽZ, TZZ, Triglav Poko...",NaN
4,Informacije,Sprememba kontaktnih podatkov,Sprememba kontaktnih podatkov,NaN,Kako uredim spremembo osebnih podatkov?,Za spremembo osebnih podatkov je potrebno odda...,NaN,"Stranki pojasnimo, da lahko vlogo za spremembo...",NaN,"naslov, pošta, sprememba, osebni podatki, zaht...",Da,"Zaledna podpora, Podpora ŽZ, TZZ, Triglav Poko...",NaN


In [5]:
nivo1 = data['ASISTENCA/INFO'].values
nivo2 = data['VRSTA STORITVE - Področje asistence oz. informacij'].values
nivo3 = data['VSEBINA VPRAŠANJA'].values
nivo3a = data['Unnamed: 3'].values
vprasanje = data['STRANKINA VPRAŠANJA Kaj stranke vprašajo pri določeni temi; Kaj jih zanima?'].values
keywords = data['KLJUČNE BESEDE'].values

In [6]:
def gpt4_translate(prompt):
    return model([HumanMessage(content=prompt)])

# Then you can use it like this:
# translation = gpt4_translate("Pozdravljen svet!")

In [38]:
question = "Kaj je Triglav komplet? In kako lahko podaljšam svojp prometno"

prompt = f"""Translate input from Slovenian to English. This is text from a Slovenian insurance company FAQ. Leave the abbreviations as they are.
Slovenian:
Kako uredim spremembo imena/priimka?
Translation to English:
How do I arrange a name/surname change?
Slovenian:
Triglav komplet.
Translation to English:
Triglav package.
Slovenian:
Drajv
Translation to English:
Drajv
Slovenian:
prometna
Translation to English:
traffic permit
Slovenian:
Torej vozniki, kateri vozijo vozila z diesel motorji več kot 20 let brez nesreč, niso upravičene do tega popusta?
Translation to English:
So drivers who have been driving diesel vehicles for more than 20 years without an accident are not eligible for this discount?
Slovenian:
AMZS
Translation to English:
AMZS

Input:
{question}
Translation:
""" 

translated_text = gpt4_translate(prompt).content

In [39]:
print(translated_text)

What is Triglav package? And how can I extend my traffic permit?


In [8]:
import time
start_time = time.time()

array = []

with open("TaxonomyGPTranslated-All_new.txt", "w") as file:
    file.write("[\n")
    for  i in range(len(nivo1)):
        tmpJson = f""""level1": "{nivo1[i]}",
"level2": "{nivo2[i]}",
"level3": "{nivo3[i]}",
"level3a": "{nivo3a[i]}",
"question": "{vprasanje[i]}",
"keywords": "{keywords[i]}" """

        prompt = f"""Translate input from Slovenian to English and put them in the right format. This is text from a Slovenian insurance company FAQ. Leave the abbreviations as they are.
Slovenian:
Kako uredim spremembo imena/priimka?
Translation to English:
How do I arrange a name/surname change?
Slovenian:
Triglav komplet.
Translation to English:
Triglav package.
Slovenian:
Drajv
Translation to English:
Drajv
Slovenian:
prometna
Translation to English:
traffic permit
Slovenian:
Torej vozniki, kateri vozijo vozila z diesel motorji več kot 20 let brez nesreč, niso upravičene do tega popusta?
Translation to English:
So drivers who have been driving diesel vehicles for more than 20 years without an accident are not eligible for this discount?
Slovenian:
AMZS
Translation to English:
AMZS

Input:
"level1": "Informacije",
"level2": "Sprememba kontaktnih podatkov",
"level3": "Sprememba kontaktnih podatkov",
"level3a": "nan",
"question": "Kako uredim spremembo imena/priimka?",
"keywords": "ime, priimek, osebni podatek, kontakt, zahtevek, sprememba" 
Formated translation: 
"level1": "Information",
"level2": "Change of contact details",
"level3": "Change of contact details",
"level3a": null,
"question": "How do I arrange a name/surname change?",
"keywords": ["name", "surname", "personal data", "contact", "request", "change"]

Input:
{tmpJson}
Formated translation:
=>
"""    
        translated_text = gpt4_translate(prompt).content
        translated_text_JSON = "{\n\"id\": " + str(i) + ",\n" + translated_text + "\n},"
        file.write(translated_text_JSON + "\n")
        array.append(translated_text_JSON)

        print(f"Input:\n{tmpJson}")
        print(f"Output:\n{translated_text}\n")

    file.write("]")

print("--- %s seconds ---" % (time.time() - start_time))

Input:
"level1": "Informacije",
"level2": "Sprememba kontaktnih podatkov",
"level3": "Sprememba kontaktnih podatkov",
"level3a": "nan",
"question": "Kako uredim spremembo imena/priimka?",
"keywords": "ime, priimek, osebni podatek, kontakt, zahtevek, sprememba" 
Output:
"level1": "Information",
"level2": "Change of contact details",
"level3": "Change of contact details",
"level3a": null,
"question": "How do I arrange a name/surname change?",
"keywords": ["name", "surname", "personal data", "contact", "request", "change"]

Input:
"level1": "Informacije",
"level2": "Sprememba kontaktnih podatkov",
"level3": "Sprememba kontaktnih podatkov",
"level3a": "nan",
"question": "Kako uredim spremembo kontaktnega podatka?",
"keywords": "osebni podatek, kontakt, zahtevek, sprememba" 
Output:
"level1": "Information",
"level2": "Change of contact details",
"level3": "Change of contact details",
"level3a": null,
"question": "How do I arrange a change of contact details?",
"keywords": ["personal data", 

In [8]:
with open("TaxonomyGPTranslated-All_SLO.txt", "w") as file:
    file.write("[\n")
    for  i in range(len(nivo1)):
        tmpJson = f""""level1": "{nivo1[i]}",
"level2": "{nivo2[i]}",
"level3": "{nivo3[i]}",
"level3a": "{nivo3a[i]}",
"question": "{vprasanje[i]}",
"keywords": "{keywords[i]}" """
        tmpJson = "{\n\"id\": " + str(i) + ",\n" + tmpJson + "\n},"
        file.write(tmpJson + "\n")
    file.write("]")

In [32]:
3966.80 / 60

66.11333333333333

In [46]:
#New wiht few shot translatiion
#For 45 rows it took 296.88823771476746 seconds / 4.948137295246124 minutes
min = 296.88823771476746 / 60
print(min)

4.9481372952461244
